# Graph Construction from High-Dimensional Vector Data
## Complete Tutorial for `linked.datasrc.vectors`

This notebook provides a comprehensive guide to building graphs from vector data using the `linked.datasrc.vectors` module. We'll cover installation, basic usage, and advanced techniques with working examples.

---

## Table of Contents

1. [Introduction & Installation](#introduction)
2. [Quick Start](#quick-start)
3. [Graph Construction Methods](#methods)
   - k-Nearest Neighbors Graph
   - Mutual k-NN Graph
   - Epsilon-Neighborhood Graph
   - Adaptive k-NN Graph
   - Random Graphs
4. [Working Examples](#examples)
5. [Performance & Scaling](#performance)
6. [Best Practices](#best-practices)
7. [Integration Examples](#integration)

---
## 1. Introduction & Installation <a id="introduction"></a>

### What is this module?

The `linked.datasrc.vectors` module provides tools to convert point/vector data into graph data (edge lists) suitable for:
- Network analysis
- Visualization (force-directed layouts)
- Dimensionality reduction
- Clustering and community detection

### Key Features

- **Scalable**: Designed for 50K-500K points with up to 3000+ dimensions
- **Fast**: Approximate methods provide 10-100x speedup
- **Flexible**: Multiple algorithms, distance metrics, and parameters
- **Research-based**: Implements algorithms from recent papers (UMAP, mutual k-NN)

### Installation

First, let's install the required dependencies:

In [1]:
# Install required packages
# Uncomment the following lines if you need to install:

# !pip install numpy scipy scikit-learn
# !pip install pynndescent  # Optional, for 10-100x speedup on large datasets

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
from typing import Tuple

# Add the linked package to the path (adjust this to your installation)
# sys.path.insert(0, '/path/to/linked')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

Libraries imported successfully!


---
## 2. Quick Start <a id="quick-start"></a>

Let's start with the simplest possible example to verify everything works.

In [5]:
# Import the main functions
from linked import knn_graph, mutual_knn_graph, adaptive_knn_graph, epsilon_graph, random_graph

print("✓ Import successful!")

✓ Import successful!


### Your First Graph (1 minute)

Let's create some sample data and build a simple k-NN graph:

In [6]:
# Create sample data: 100 points in 20 dimensions
vectors = np.random.rand(100, 20)
print(f"Created {vectors.shape[0]} vectors with {vectors.shape[1]} dimensions")

# Build a k-NN graph (each point connected to 5 nearest neighbors)
graph = knn_graph(vectors, n_neighbors=5)

print(f"\nCreated graph with {len(graph)} edges")
print(f"Graph shape: {graph.shape}")
print(f"\nFirst 5 edges (source, target, weight):")
print(graph[:5])

Created 100 vectors with 20 dimensions

Created graph with 500 edges
Graph shape: (500, 3)

First 5 edges (source, target, weight):
[[ 0.         76.          1.22005713]
 [ 0.         46.          1.24272346]
 [ 0.         63.          1.27833676]
 [ 0.         42.          1.34552133]
 [ 0.         94.          1.39535344]]


### Understanding the Output

The graph is returned as a numpy array where:
- **Column 0**: Source node index
- **Column 1**: Target node index  
- **Column 2**: Edge weight (distance between nodes)

For example, `[0, 15, 1.234]` means:
- Node 0 is connected to node 15
- The distance between them is 1.234

---
## 3. Graph Construction Methods <a id="methods"></a>

The module provides five different methods for constructing graphs from vector data. Let's explore each one.

### 3.1 k-Nearest Neighbors (k-NN) Graph

The most basic approach: connect each point to its k nearest neighbors.

**When to use:**
- Standard graph construction
- Low to medium dimensional data
- When you want a simple, predictable structure

In [ ]:
print("="*60)
print("Example 1: Basic k-NN Graph")
print("="*60)

# Generate synthetic data
n_samples = 200
n_features = 32
vectors = np.random.rand(n_samples, n_features)
print(f"Data shape: {vectors.shape}")

# Build k-NN graph
graph = knn_graph(
    vectors,
    n_neighbors=10,        # Each point connects to 10 nearest neighbors
    metric="euclidean",    # Distance metric
    mode="distance",       # Include distance weights
    approximate=False      # Use exact nearest neighbors
)

print(f"\nGraph statistics:")
print(f"  Number of edges: {len(graph)}")
print(f"  Average edge weight: {np.mean(graph[:, 2]):.4f}")
print(f"  Min edge weight: {np.min(graph[:, 2]):.4f}")
print(f"  Max edge weight: {np.max(graph[:, 2]):.4f}")

print(f"\nSample edges:")
print(graph[:5])

#### Comparing Distance Metrics

Different metrics are appropriate for different types of data:

In [ ]:
# Compare different distance metrics
vectors = np.random.rand(100, 32)
metrics = ["euclidean", "manhattan", "cosine"]

print("\nComparing distance metrics:")
print("-" * 60)

for metric in metrics:
    graph = knn_graph(vectors, n_neighbors=10, metric=metric, approximate=False)
    avg_weight = np.mean(graph[:, 2])
    print(f"{metric.capitalize():12s}: {len(graph):4d} edges, avg distance = {avg_weight:.4f}")

**Metric Selection Guide:**
- **Euclidean**: General-purpose, works well for normalized data
- **Cosine**: Good for high-dimensional sparse data (e.g., text embeddings)
- **Manhattan**: Robust to outliers, good for discrete features

### 3.2 Mutual k-NN Graph

A more robust variant where an edge exists only if two points are mutual neighbors.

**When to use:**
- High-dimensional data (reduces "hub" effects)
- When you want more reliable local structure
- Visualization tasks requiring cleaner structure

In [ ]:
print("="*60)
print("Example 2: Mutual k-NN Graph")
print("="*60)

vectors = np.random.rand(150, 64)
print(f"Data shape: {vectors.shape}")

# Regular k-NN graph
graph_regular = knn_graph(vectors, n_neighbors=10, approximate=False)
print(f"\nRegular k-NN edges: {len(graph_regular)}")

# Mutual k-NN graph (no connectivity enforcement)
graph_mutual = mutual_knn_graph(
    vectors,
    n_neighbors=10,
    approximate=False,
    ensure_connectivity="none"  # Don't add extra edges for connectivity
)
print(f"Mutual k-NN edges: {len(graph_mutual)}")
print(f"Sparsity reduction: {len(graph_mutual)/len(graph_regular)*100:.1f}%")

# Mutual k-NN with MST connectivity (recommended)
graph_mutual_mst = mutual_knn_graph(
    vectors,
    n_neighbors=10,
    approximate=False,
    ensure_connectivity="mst"  # Add minimum spanning tree edges
)
print(f"Mutual k-NN + MST edges: {len(graph_mutual_mst)}")
print(f"\nMST adds {len(graph_mutual_mst) - len(graph_mutual)} edges to ensure connectivity")

**Why Mutual k-NN is Better for High Dimensions:**

In high dimensions, some points become "hubs" with many incoming connections. Mutual k-NN removes these asymmetric connections, resulting in:
- More balanced degree distribution
- Better preservation of local structure
- Cleaner visualization

### 3.3 Epsilon-Neighborhood Graph

Connect all pairs of points within a fixed radius.

**When to use:**
- When you have a natural distance threshold
- For density-based analysis
- When point density varies significantly

In [ ]:
print("="*60)
print("Example 3: Epsilon-Neighborhood Graph")
print("="*60)

vectors = np.random.rand(100, 16)
print(f"Data shape: {vectors.shape}")

# Try different radii to see effect
print("\nTesting different radius values:")
print("-" * 40)

for radius in [0.3, 0.5, 0.7, 1.0]:
    graph = epsilon_graph(vectors, radius=radius)
    avg_degree = len(graph) / len(vectors) if len(graph) > 0 else 0
    print(f"Radius {radius:.1f}: {len(graph):4d} edges, avg degree = {avg_degree:.2f}")

**Note:** Epsilon graphs can be very sensitive to the radius parameter. Too small and the graph fragments; too large and it becomes dense.

### 3.4 Adaptive k-NN Graph (UMAP-style)

Uses local density estimation to adaptively scale distances, ensuring consistent local connectivity across varying-density regions.

**When to use:**
- Data with varying density regions
- For dimensionality reduction preprocessing
- When you want smooth, adaptive weights

In [ ]:
print("="*60)
print("Example 4: Adaptive k-NN Graph (UMAP-style)")
print("="*60)

# Create data with varying density: two clusters with different spreads
cluster1 = np.random.randn(50, 10) * 0.3 + 0  # Dense cluster
cluster2 = np.random.randn(50, 10) * 1.0 + 5  # Sparse cluster
vectors = np.vstack([cluster1, cluster2])

print(f"Data shape: {vectors.shape}")
print("Two clusters with different densities\n")

# Standard k-NN
graph_standard = knn_graph(vectors, n_neighbors=8, approximate=False)

# Adaptive k-NN
graph_adaptive = adaptive_knn_graph(
    vectors,
    n_neighbors=8,
    local_connectivity=1,  # Min strong connections per point
    bandwidth=1.0,         # Gaussian kernel bandwidth
    approximate=False
)

print("Comparison:")
print("-" * 60)
print(f"Standard k-NN:  {len(graph_standard)} edges")
print(f"Adaptive k-NN:  {len(graph_adaptive)} edges")

print(f"\nWeight distributions:")
print(f"Standard - mean: {np.mean(graph_standard[:, 2]):.4f}, std: {np.std(graph_standard[:, 2]):.4f}")
print(f"Adaptive - mean: {np.mean(graph_adaptive[:, 2]):.4f}, std: {np.std(graph_adaptive[:, 2]):.4f}")

print("\nNote: Adaptive weights are in (0, 1] after exponential transformation")

**How Adaptive Weighting Works:**

1. Estimates local density around each point
2. Scales distances by local density
3. Applies Gaussian kernel for smooth weights
4. Result: Consistent connectivity regardless of local density

### 3.5 Random Graphs

Generate synthetic graphs for testing and benchmarking.

**Available models:**
- **Erdős-Rényi**: Random edges with fixed probability
- **Barabási-Albert**: Scale-free network with preferential attachment
- **Watts-Strogatz**: Small-world network with local clustering

In [ ]:
print("="*60)
print("Example 5: Random Graph Generation")
print("="*60)

n_nodes = 50

# Erdős-Rényi: Each edge exists with probability p
graph_er = random_graph(n_nodes, model="erdos_renyi", p=0.1, seed=42)
print(f"Erdős-Rényi (p=0.1):         {len(graph_er):3d} edges")

# Barabási-Albert: Preferential attachment (m edges per new node)
graph_ba = random_graph(n_nodes, model="barabasi_albert", m=2, seed=42)
print(f"Barabási-Albert (m=2):       {len(graph_ba):3d} edges")

# Watts-Strogatz: Small-world (k initial degree, p rewiring probability)
graph_ws = random_graph(n_nodes, model="watts_strogatz", k=4, p=0.2, seed=42)
print(f"Watts-Strogatz (k=4, p=0.2): {len(graph_ws):3d} edges")

# With random weights
graph_weighted = random_graph(
    n_nodes,
    model="erdos_renyi",
    p=0.1,
    weighted=True,
    seed=42
)
print(f"\nWeighted Erdős-Rényi: mean weight = {np.mean(graph_weighted[:, 2]):.4f}")

---
## 4. Working Examples <a id="examples"></a>

Now let's look at more complete, real-world examples.

### Example A: Processing Text Embeddings

A common use case is building a graph from text embeddings (e.g., word2vec, BERT, etc.).

In [ ]:
print("="*60)
print("Example A: Text Embeddings")
print("="*60)

# Simulate text embeddings (e.g., from BERT)
n_documents = 500
embedding_dim = 768  # Typical BERT embedding dimension

# Generate random embeddings (in practice, these would come from your model)
text_embeddings = np.random.randn(n_documents, embedding_dim)
# Normalize for cosine similarity
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

print(f"Processing {n_documents} documents with {embedding_dim}-dim embeddings")

# Build graph using cosine similarity (appropriate for text)
start_time = time.time()
graph = knn_graph(
    text_embeddings,
    n_neighbors=20,
    metric="cosine",       # Cosine distance for text
    approximate=True,      # Fast approximate search
    n_jobs=-1             # Use all CPU cores
)
elapsed = time.time() - start_time

print(f"\nGraph construction completed in {elapsed:.3f} seconds")
print(f"Created {len(graph)} edges")
print(f"Average cosine distance: {np.mean(graph[:, 2]):.4f}")
print(f"\nThis graph can be used for:")
print("  - Document similarity search")
print("  - Topic clustering")
print("  - Visualization of document relationships")

### Example B: Creating Clusters with Different Structures

Let's create synthetic data with clear cluster structure and see how different methods handle it.

In [ ]:
print("="*60)
print("Example B: Clustered Data")
print("="*60)

# Create 3 distinct clusters
n_per_cluster = 100
cluster_centers = np.array([[0, 0], [5, 5], [-5, 5]])

clusters = []
for center in cluster_centers:
    cluster = np.random.randn(n_per_cluster, 2) * 0.5 + center
    clusters.append(cluster)

vectors = np.vstack(clusters)
print(f"Created {len(vectors)} points in 3 clusters")

# Build mutual k-NN graph with MST connectivity
graph = mutual_knn_graph(
    vectors,
    n_neighbors=10,
    ensure_connectivity="mst"
)

print(f"\nGraph has {len(graph)} edges")

# Analyze intra-cluster vs inter-cluster edges
labels = np.repeat(np.arange(3), n_per_cluster)

intra_cluster = 0
inter_cluster = 0

for source, target, weight in graph:
    if labels[int(source)] == labels[int(target)]:
        intra_cluster += 1
    else:
        inter_cluster += 1

print(f"\nEdge analysis:")
print(f"  Intra-cluster edges: {intra_cluster} ({intra_cluster/len(graph)*100:.1f}%)")
print(f"  Inter-cluster edges: {inter_cluster} ({inter_cluster/len(graph)*100:.1f}%)")
print(f"\nGood graph construction preserves cluster structure!")

### Example C: Using Sklearn-Style Estimators

The module provides sklearn-compatible estimators for pipeline integration.

In [ ]:
print("="*60)
print("Example C: Sklearn-Style Estimators")
print("="*60)

from linked import KNNGraphEstimator, MutualKNNGraphEstimator, AdaptiveKNNGraphEstimator

# Sample data
vectors = np.random.rand(200, 50)

print("Creating estimators with different configurations:\n")

# Standard k-NN estimator
est_knn = KNNGraphEstimator(
    n_neighbors=15,
    metric="euclidean",
    approximate=True
)
graph_knn = est_knn.fit_transform(vectors)
print(f"KNN Estimator:           {len(graph_knn)} edges")

# Mutual k-NN estimator
est_mutual = MutualKNNGraphEstimator(
    n_neighbors=15,
    ensure_connectivity="mst"
)
graph_mutual = est_mutual.fit_transform(vectors)
print(f"Mutual KNN Estimator:    {len(graph_mutual)} edges")

# Adaptive k-NN estimator
est_adaptive = AdaptiveKNNGraphEstimator(
    n_neighbors=15,
    local_connectivity=1,
    bandwidth=1.0
)
graph_adaptive = est_adaptive.fit_transform(vectors)
print(f"Adaptive KNN Estimator:  {len(graph_adaptive)} edges")

print("\nEstimators can be used in sklearn pipelines!")

---
## 5. Performance & Scaling <a id="performance"></a>

Let's examine performance characteristics with different dataset sizes.

### 5.1 Comparing Exact vs Approximate Methods

In [ ]:
print("="*60)
print("Performance Comparison: Exact vs Approximate")
print("="*60)

# Generate larger dataset
n_samples = 5000
n_features = 256
vectors = np.random.rand(n_samples, n_features)

print(f"Testing with {n_samples} samples, {n_features} features\n")

# Exact k-NN (slower but precise)
print("Running exact k-NN...")
start = time.time()
graph_exact = knn_graph(
    vectors,
    n_neighbors=15,
    approximate=False,
    n_jobs=-1
)
time_exact = time.time() - start
print(f"  Time: {time_exact:.3f} seconds")
print(f"  Edges: {len(graph_exact)}")

# Approximate k-NN (faster)
print("\nRunning approximate k-NN...")
try:
    start = time.time()
    graph_approx = knn_graph(
        vectors,
        n_neighbors=15,
        approximate=True,
        n_jobs=-1
    )
    time_approx = time.time() - start
    print(f"  Time: {time_approx:.3f} seconds")
    print(f"  Edges: {len(graph_approx)}")
    print(f"\n  Speedup: {time_exact/time_approx:.1f}x")
except Exception as e:
    print(f"  Approximate method not available (install pynndescent for speedup)")
    print(f"  Would provide ~10-100x speedup!")

### 5.2 Scaling Guidelines

Here are recommended settings for different dataset sizes:

| Dataset Size | Recommended Settings |
|--------------|---------------------|
| < 1K samples | `approximate=False`, any metric |
| 1K - 10K | `approximate=True`, `n_neighbors=15` |
| 10K - 100K | `approximate=True`, `n_neighbors=10-15` |
| 100K - 500K | `approximate=True`, `n_neighbors=5-10` |
| > 500K | Consider batch processing |

**Memory Requirements:**

For 100K samples × 256 features with k=15:
- Input vectors: ~200 MB
- Output graph: ~36 MB
- Intermediate: ~400-600 MB
- **Total: ~0.6-1 GB**

### 5.3 Optimizing Parameters

In [ ]:
print("="*60)
print("Parameter Optimization Example")
print("="*60)

vectors = np.random.rand(1000, 128)

# Test different n_neighbors values
print("\nEffect of n_neighbors on graph density:")
print("-" * 60)

for k in [5, 10, 15, 20, 30]:
    graph = knn_graph(vectors, n_neighbors=k, approximate=False)
    avg_degree = len(graph) / len(vectors)
    avg_weight = np.mean(graph[:, 2])
    print(f"k={k:2d}: {len(graph):5d} edges, avg_degree={avg_degree:.1f}, avg_weight={avg_weight:.4f}")

print("\nGeneral guidelines:")
print("  - Smaller k (5-10): Sparse, emphasize local structure")
print("  - Medium k (15-30): Balanced, good for most cases")
print("  - Larger k (50+):   Dense, may include noise")

---
## 6. Best Practices <a id="best-practices"></a>

Here are some guidelines for getting the best results.

### 6.1 Data Preprocessing

**Always normalize or standardize your data** before graph construction:

In [ ]:
print("="*60)
print("Data Preprocessing Example")
print("="*60)

# Raw data with different scales
raw_data = np.random.rand(200, 10)
raw_data[:, 0] *= 100  # First feature has much larger scale

print("Raw data statistics:")
print(f"  Feature 0: mean={np.mean(raw_data[:, 0]):.2f}, std={np.std(raw_data[:, 0]):.2f}")
print(f"  Feature 1: mean={np.mean(raw_data[:, 1]):.2f}, std={np.std(raw_data[:, 1]):.2f}")

# Method 1: Standardization (zero mean, unit variance)
standardized_data = (raw_data - np.mean(raw_data, axis=0)) / np.std(raw_data, axis=0)

# Method 2: Min-Max normalization (scale to [0, 1])
min_vals = np.min(raw_data, axis=0)
max_vals = np.max(raw_data, axis=0)
normalized_data = (raw_data - min_vals) / (max_vals - min_vals)

# Method 3: L2 normalization (for cosine similarity)
l2_normalized_data = raw_data / np.linalg.norm(raw_data, axis=1, keepdims=True)

# Compare graph construction
graph_raw = knn_graph(raw_data, n_neighbors=10, approximate=False)
graph_standardized = knn_graph(standardized_data, n_neighbors=10, approximate=False)
graph_normalized = knn_graph(normalized_data, n_neighbors=10, approximate=False)

print("\nAverage edge weights:")
print(f"  Raw data:         {np.mean(graph_raw[:, 2]):.4f}")
print(f"  Standardized:     {np.mean(graph_standardized[:, 2]):.4f}")
print(f"  Min-Max norm:     {np.mean(graph_normalized[:, 2]):.4f}")

print("\nRecommendation: Standardize or normalize before graph construction!")

### 6.2 Choosing the Right Method

**Decision Tree for Method Selection:**

```
Start
│
├─ High-dimensional data (>100 dims)?
│  └─ YES → Use mutual_knn_graph with ensure_connectivity="mst"
│
├─ Varying density regions?
│  └─ YES → Use adaptive_knn_graph
│
├─ Natural distance threshold?
│  └─ YES → Use epsilon_graph
│
└─ Standard case → Use knn_graph
```

### 6.3 Common Pitfalls to Avoid

In [ ]:
print("="*60)
print("Common Pitfalls and Solutions")
print("="*60)

print("""
❌ PITFALL 1: Using k too large
   Problem: Graph becomes too dense, includes noise
   Solution: Start with k=15, increase only if needed

❌ PITFALL 2: Not normalizing data
   Problem: Features with large scales dominate distance
   Solution: Always standardize or normalize first

❌ PITFALL 3: Using Euclidean for high-D sparse data
   Problem: Distance concentration makes neighbors less meaningful
   Solution: Use cosine metric for sparse/text data

❌ PITFALL 4: Forgetting connectivity
   Problem: Mutual k-NN can fragment into disconnected components
   Solution: Use ensure_connectivity="mst"

❌ PITFALL 5: Using exact methods on large data
   Problem: Very slow for >10K points
   Solution: Set approximate=True, install pynndescent

✅ BEST PRACTICE: Start simple, iterate based on results
""")

---
## 7. Integration Examples <a id="integration"></a>

Let's see how to integrate the graphs with other tools.

### 7.1 Converting to NetworkX

NetworkX is a popular Python library for network analysis:

In [ ]:
print("="*60)
print("Integration: NetworkX")
print("="*60)

try:
    import networkx as nx
    
    # Create graph
    vectors = np.random.rand(50, 10)
    edges = knn_graph(vectors, n_neighbors=5)
    
    # Convert to NetworkX
    G = nx.Graph()
    
    # Add edges with weights
    for source, target, weight in edges:
        G.add_edge(int(source), int(target), weight=weight)
    
    print(f"Created NetworkX graph with:")
    print(f"  Nodes: {G.number_of_nodes()}")
    print(f"  Edges: {G.number_of_edges()}")
    print(f"  Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
    
    # Network analysis
    print(f"\nNetwork metrics:")
    print(f"  Connected: {nx.is_connected(G)}")
    if nx.is_connected(G):
        print(f"  Diameter: {nx.diameter(G):.2f}")
        print(f"  Avg path length: {nx.average_shortest_path_length(G):.2f}")
    print(f"  Clustering coefficient: {nx.average_clustering(G):.4f}")
    
except ImportError:
    print("NetworkX not installed. Install with: pip install networkx")
    print("\nExample code:")
    print("""
    import networkx as nx
    
    G = nx.Graph()
    for source, target, weight in edges:
        G.add_edge(int(source), int(target), weight=weight)
    """)

### 7.2 Visualization with Force-Directed Layout

Create a simple 2D visualization:

In [ ]:
print("="*60)
print("Integration: Visualization")
print("="*60)

try:
    import networkx as nx
    import matplotlib.pyplot as plt
    
    # Create small graph for visualization
    vectors = np.random.rand(30, 5)
    edges = mutual_knn_graph(vectors, n_neighbors=4, ensure_connectivity="mst")
    
    # Convert to NetworkX
    G = nx.Graph()
    for source, target, weight in edges:
        G.add_edge(int(source), int(target), weight=weight)
    
    # Create layout
    pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)
    
    # Plot
    plt.figure(figsize=(10, 8))
    
    # Draw nodes
    nx.draw_networkx_nodes(
        G, pos,
        node_color='lightblue',
        node_size=500,
        alpha=0.9
    )
    
    # Draw edges with varying thickness based on weight
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    max_weight = max(weights)
    edge_widths = [3 * (1 - w/max_weight) for w in weights]  # Thicker = closer
    
    nx.draw_networkx_edges(
        G, pos,
        width=edge_widths,
        alpha=0.5,
        edge_color='gray'
    )
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    plt.title('Graph Visualization: Mutual k-NN with MST', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Visualization created successfully!")
    
except ImportError:
    print("NetworkX or matplotlib not installed")
    print("Install with: pip install networkx matplotlib")

### 7.3 Using Graphs for Clustering

In [ ]:
print("="*60)
print("Integration: Graph-Based Clustering")
print("="*60)

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

# Create data with distinct clusters
cluster1 = np.random.randn(50, 10) * 0.5 + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cluster2 = np.random.randn(50, 10) * 0.5 + [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
cluster3 = np.random.randn(50, 10) * 0.5 + [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
vectors = np.vstack([cluster1, cluster2, cluster3])

print(f"Data: {len(vectors)} points in 3 clusters\n")

# Build graph
edges = mutual_knn_graph(vectors, n_neighbors=8, ensure_connectivity="none")

# Convert to sparse adjacency matrix
n_nodes = len(vectors)
row = edges[:, 0].astype(int)
col = edges[:, 1].astype(int)
data = np.ones(len(edges))
adj_matrix = csr_matrix((data, (row, col)), shape=(n_nodes, n_nodes))

# Make symmetric
adj_matrix = adj_matrix + adj_matrix.T

# Find connected components (clusters)
n_clusters, labels = connected_components(adj_matrix, directed=False)

print(f"Graph-based clustering results:")
print(f"  Number of clusters found: {n_clusters}")

for i in range(n_clusters):
    cluster_size = np.sum(labels == i)
    print(f"  Cluster {i}: {cluster_size} points")

print("\n✓ Graph structure reveals natural clusters!")

---
## Summary & Key Takeaways

### Main Functions

| Function | Use Case | Key Parameters |
|----------|----------|----------------|
| `knn_graph` | General purpose | `n_neighbors=15` |
| `mutual_knn_graph` | High dimensions | `ensure_connectivity="mst"` |
| `epsilon_graph` | Distance threshold | `radius=0.5` |
| `adaptive_knn_graph` | Varying density | `local_connectivity=1` |
| `random_graph` | Testing/benchmarking | `model="erdos_renyi"` |

### Quick Reference

```python
from linked import knn_graph, mutual_knn_graph

# Basic usage
graph = knn_graph(vectors, n_neighbors=15)

# For high-dimensional data
graph = mutual_knn_graph(
    vectors,
    n_neighbors=15,
    ensure_connectivity="mst"
)

# For large datasets
graph = knn_graph(
    vectors,
    n_neighbors=15,
    approximate=True,
    n_jobs=-1
)
```

### Remember

1. **Always preprocess**: Normalize or standardize your data
2. **Start simple**: Begin with `knn_graph`, iterate if needed
3. **Use approximation**: Set `approximate=True` for datasets >1K points
4. **Choose metric wisely**: Cosine for text, Euclidean for general data
5. **Ensure connectivity**: Use MST for visualization tasks

### Next Steps

- Read the [API Reference](linked/misc/API_REFERENCE.md) for complete documentation
- Check [examples.py](linked/misc/examples.py) for more advanced usage
- Explore the [README](linked/README.md) for detailed explanations

---

**Happy graphing! 🎯**